In [7]:
import time,cv2,os,glob,random
os.sys.path.append('../utilities/')
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [3]:
model_path = "/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/ssd_mobilenet_v1_coco/ssd_mobilenet_v1_coco_2018_01_28/openvino_model/frozen_inference_graph.xml"

In [8]:
output_support = support_utility_openvino.create_plot()

In [5]:
ssd_openvino_model = support_utility_openvino.async_infer(4)
ssd_openvino_model.load_model(model_path,"CPU")

Available Devices :  ['CPU', 'GNA', 'GPU']
OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [8]:
channel = "/media/prateek/shared_space/DATA_N_MODELS/useful_vids/general/vehicle/4K camera example for Traffic Monitoring (Road)-jjlBnrzSGjc.mp4"
thresh = .5
cap = cv2.VideoCapture(channel)
cap.set(1,500)
ret,frame = cap.read()

res_thresh = output_support.get_resolution_thresh(frame)

cv2.namedWindow("openvino",cv2.WINDOW_NORMAL)
ssd_openvino_model.reinit_model()

write_video = False
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("../output_vids/general/ssd/"+"output_ssd_openvino"+".mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))
while True:
#     Disp.clear_output(wait=True)
    fps = fps[-100:]
#     for i in range(100):
    ret,frame = cap.read()
    if not ret:
        break

    t1 = time.time()
    ssd_openvino_model.predict(frame)
    if ssd_openvino_model.frame_processed>=ssd_openvino_model.num_requests:
        frame,attr,res = ssd_openvino_model.postprocess_op()       
        fps.append(1/(time.time()-t1))
        
        op_frame =  output_support.label_coco(frame,res[0],thresh=thresh,resolution_thresh_range=res_thresh)
        

        op_frame = output_support.write_text(op_frame,"FPS = "+str(np.mean(fps,dtype='int')),text_color=(0, 0, 0))
        if write_video:
            out.write(op_frame)
        cv2.imshow("preview",op_frame)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
if write_video:
    out.release()
cap.release()
cv2.destroyAllWindows()

                
            

    
    

In [9]:

import tensorflow as tf
import numpy as np
import cv2,time
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

# Read the graph.

with tf.gfile.FastGFile('/media/prateek/shared_space/DATA_N_MODELS/useful_models/openvino/public/ssd_mobilenet_v1_coco/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

channel = "/media/prateek/shared_space/DATA_N_MODELS/useful_vids/general/vehicle/4K camera example for Traffic Monitoring (Road)-jjlBnrzSGjc.mp4"
thresh = .5
colors = dict(zip(range(1,81),np.random.randint(0,255,(80,3),dtype = np.int16)))
cap = cv2.VideoCapture(channel)
cap.set(1,500)

fps = []
write_video = False
if write_video:
    fps = fps[-100:]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_vid = cv2.VideoWriter("../output_vids/general/ssd/"+"output_ssd_tensorflow"+".mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))

with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    # Read and preprocess an image.
    while True:
        ret,img  = cap.read()
        if not ret:
            break
        rows = img.shape[0]
        cols = img.shape[1]
        inp = cv2.resize(img, (300, 300))
#         inp = inp[:, :, [2, 1, 0]]  # BGR2RGB
        t1 = time.time()
        # Run the model
        out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                        sess.graph.get_tensor_by_name('detection_scores:0'),
                        sess.graph.get_tensor_by_name('detection_boxes:0'),
                        sess.graph.get_tensor_by_name('detection_classes:0')],
                       feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
        fps.append(1/(time.time()-t1))

        # Visualize detected bounding boxes.
        num_detections = int(out[0][0])
        for i in range(num_detections):
            classId = int(out[3][0][i])
            color = tuple(colors[classId])
            score = float(out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            if score > thresh:
                x = bbox[1] * cols
                y = bbox[0] * rows
                right = bbox[3] * cols
                bottom = bbox[2] * rows
                cv2.rectangle(img, (int(x), int(y)), (int(right), int(bottom)),tuple(map(int,color)), thickness=2)
        img = output_support.write_text(img,"FPS = "+str(np.mean(fps,dtype='int')),text_color=(0, 0, 0))
        if write_video:
            out_vid.write(img)
        cv2.imshow("preview",img)
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
if write_video:
    out_vid.release()